In [111]:
import requests
import csv
import pandas as pd

In [112]:
payload = {'limit': '10',
           'q': 't_1h: [2021-12-12T23:00:00Z TO 2021-12-14T22:59:59Z]',
           'sort': '-t_1h',
           'facet':'libelle',
           'facet': 't_1h',
           'facet': 'etat_trafic',
           'facet': 'libelle_nd_amont',
           'facet': 'libelle_nd_aval'}

res1 = requests.get('https://opendata.paris.fr/api/v2/catalog/datasets/comptages-routiers-permanents/exports/csv/',
                  params=payload)

decoded_content = res1.content.decode('utf-8')

cr = csv.reader(decoded_content.splitlines(), delimiter=';')
my_list = list(cr)
df = pd.DataFrame(my_list)
df = df.rename(columns=df.iloc[0]).drop(0)
df = df.reset_index(drop=True)

In [113]:
df

,iu_ac,libelle,t_1h,q,k,etat_trafic,iu_nd_amont,libelle_nd_amont,iu_nd_aval,libelle_nd_aval,etat_barre,date_debut,date_fin,geo_point_2d,geo_shape
0,4541,Bd_Lannes,2021-12-12T23:00:00+00:00,81.0,0.31778,Fluide,2423,Bd_Lannes-Longchamp,2422,Lannes-Piscine,Invalide,2005-01-01,2019-06-01,"48.8692906671, 2.2737212675","{""type"": ""LineString"", ""coordinates"": [[2.2741..."
1,4537,Bd_Lannes,2021-12-12T23:00:00+00:00,210.0,0.55222,Fluide,2421,Lannes-Dufrenoy,2422,Lannes-Piscine,Invalide,2005-01-01,2019-06-01,"48.8676607046, 2.27260371129","{""type"": ""LineString"", ""coordinates"": [[2.2716..."
2,4538,Bd_Lannes,2021-12-12T23:00:00+00:00,210.0,,Inconnu,2422,Lannes-Piscine,2423,Bd_Lannes-Longchamp,Invalide,2005-01-01,2019-06-01,"48.8692523427, 2.27390750662","{""type"": ""LineString"", ""coordinates"": [[2.2735..."
3,4563,Souterrain_Varsovie_VGP,2021-12-12T23:00:00+00:00,659.0,6.91389,Fluide,2428,VGP-Niveau_Le_Notre,2425,Av_de_New_York-Albert_de_Mun,Invalide,2005-01-01,2019-06-01,"48.860692223, 2.29155031553","{""type"": ""LineString"", ""coordinates"": [[2.2895..."
4,4572,Av_de_New_York,2021-12-12T23:00:00+00:00,,3.52167,Fluide,2428,VGP-Niveau_Le_Notre,2426,Pl_de_Varsovie,Invalide,2005-01-01,2019-06-01,"48.8596673662, 2.29031932381","{""type"": ""LineString"", ""coordinates"": [[2.2895..."
5,4547,Souterrain_Varsovie_VGP,2021-12-12T23:00:00+00:00,510.0,,Inconnu,2425,Av_de_New_York-Albert_de_Mun,2428,VGP-Niveau_Le_Notre,Invalide,2005-01-01,2019-06-01,"48.8607374804, 2.29146327638","{""type"": ""LineString"", ""coordinates"": [[2.2934..."
6,4561,Av_de_New_York,2021-12-12T23:00:00+00:00,659.0,6.91389,Fluide,2417,Av_Pdt_Kennedy-Beethoven,2428,VGP-Niveau_Le_Notre,Invalide,2005-01-01,2019-06-01,"48.8585731768, 2.28890296053","{""type"": ""LineString"", ""coordinates"": [[2.2882..."
7,4562,Bd_Suchet,2021-12-12T23:00:00+00:00,345.0,,Inconnu,2430,Suchet-Boilly,2429,Suchet-Raphael,Invalide,2005-01-01,2019-06-01,"48.8605642445, 2.26693236288","{""type"": ""LineString"", ""coordinates"": [[2.2661..."
8,4548,Bd_Suchet,2021-12-12T23:00:00+00:00,102.0,0.71611,Fluide,2410,Place_de_Colombie,2429,Suchet-Raphael,Invalide,2005-01-01,2019-06-01,"48.8624292022, 2.26788320191","{""type"": ""LineString"", ""coordinates"": [[2.2682..."
9,4549,Bd_Suchet,2021-12-12T23:00:00+00:00,102.0,0.71611,Fluide,2429,Suchet-Raphael,2430,Suchet-Boilly,Invalide,2005-01-01,2019-06-01,"48.8606885864, 2.26670434794","{""type"": ""LineString"", ""coordinates"": [[2.2675..."
